# In this notebook we look into the performance of semi-dual damped Newton in the numpy framework.

In [ ]:
!pip install jaxlinop

In [ ]:
from __future__ import division
import os
import numpy as np
import time
import matplotlib.pyplot as plt
import scipy as scp
import warnings
warnings.filterwarnings('ignore')
np.random.seed(1234)
import computational_OT
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok=True)
if not os.path.isdir("../Images/DampedNewton_SemiDual_images_numpy"):
    os.makedirs("../Images/DampedNewton_SemiDual_images_numpy")

In [ ]:
"""To compute distance matrix"""
def distmat(x,y):
    return np.sum( x**2,0 )[:,None] + np.sum( y**2,0 )[None,:] - 2*x.transpose().dot(y)

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP(u,K,v):
    return u[:,None]*K*v[None,:]

def plotp(x, col,plt, scale=200, edgecolors="k"):
  return plt.scatter( x[0,:], x[1,:], s=scale, edgecolors=edgecolors,  c=col, cmap='plasma', linewidths=2 )

In [ ]:
def generate_data(N):
    """
     N is a list of the size of the data on x and y
    """
    x = np.random.rand( 2,N[0] )-0.5
    theta = 2*np.pi*np.random.rand( 1,N[1] )
    r = 0.8+.2*np.random.rand( 1,N[1] )
    y = np.vstack( ( r*np.cos( theta ),r*np.sin( theta ) ) )
    return x,y

In [ ]:
N = [ 500,  600 ]
x , y = generate_data(N)

## Entropy regularized dual-formulation
The dual formulation of the entropy regularized OT is given by:
$$
OT_{\epsilon}(\alpha,\beta) = \max_{f\in \mathbb{R}^{n}, g\in\mathbb{R}^{m}} \langle f, \alpha \rangle + \langle g, \beta \rangle - \epsilon\left(\langle\alpha \otimes \beta, e^{\frac{f}{\epsilon}}\odot K \odot e^{\frac{g}{\epsilon}}  \rangle-1\right)\ ,
$$
where
$$
\alpha \in \mathcal{M}_{1}(\mathcal{X}),\ \beta \in \mathcal{M}_{1}(\mathcal{Y}),\ \epsilon>0,\ f\in\mathbb{R}^{n},\ g\in \mathbb{R}^{m}\ .
$$

## The dual formulation of OT is given by:
Using the Shrodinger-bridge equations between the potentials, that is, $g_{j} = -\epsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}-C_{ij}}{\epsilon}\right)\alpha_{i}\right)\ , \ \forall j = 1,\dots,m$, we obtain the semi-dual formulation of the objective function, that is,
$$
Q_{\epsilon}^{semi}(f) = \langle f, \alpha \rangle + \langle g(f,C,\epsilon), \beta \rangle\ , 
$$
where
$g(f,C,\epsilon)_{j} = -\epsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}-C_{ij}}{\epsilon}\right)\alpha_{i}\right)$.

In this setup, the gradients and the Hessian is as follows:
$$
\nabla_{f}Q_{\epsilon}^{semi}(f)_{i} = \frac{1}{\epsilon}\alpha_{i}\left(1-\sum_{s=1}^{n}\frac{e^{\frac{f_{i}-C_{ij}}{\epsilon}}\beta_{s}}{\left(\sum_{t=1}^{n}\alpha_{t}e^{\frac{f_{t}-C_{ts}}{\epsilon}}\right)}\right)\ ,\ \forall i = 1,\dots,n 
$$
and
$$
\nabla^{2}_{f}Q_{\epsilon}^{semi}(f)_{ii} = \frac{-1}{\epsilon}\sum_{s=1}^{m}\left(\alpha_{i}\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\left(1 - \alpha_{i}\left(\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\right)\beta_{s}\ ,\ \forall i =1,\dots,n,
$$
and
$$
\nabla^{2}_{f}Q_{\epsilon}^{semi}(f)_{ij} = \frac{1}{\epsilon}\sum_{s=1}^{m}\alpha_{i}\alpha_{j}\left(\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\left(\exp\left(\frac{f_{j}+g(f,C,\epsilon)_{s}-C_{js}}{\epsilon}\right)\right)\beta_{s}\ ,\ \forall i \neq j = 1,\dots,n\ .
$$
Now we plug-in these gradients and Hessian in damped Newton algorithm as we did before.

Here we also use the exp-log stabilization to stabilize $g$, the gradients as well as the Hessian as below
$$
m(f)_{j} \leftarrow \min_{i}(C_{ij}-f_{i})\ ,  \ \forall j = 1,\dots,m\\
g_{j} = -\epsilon\log\left(\sum_{i}\exp\left(\frac{f_{i}-C_{ij}+m(f)_{j}}{\epsilon}\right)\alpha_{i}\right)+m(f)_{j}\ ,  \ \forall j = 1,\dots,m\ ,\\
\nabla_{f}Q_{\epsilon}^{semi}(f)_{i} = \frac{1}{\epsilon}\alpha_{i}\left(1-\sum_{s=1}^{n}\frac{e^{\frac{f_{i}-C_{ij}+m(f)_{j}}{\epsilon}}\beta_{s}}{\left(\sum_{t=1}^{n}\alpha_{t}e^{\frac{f_{t}-C_{ts}+m(f)_{j}}{\epsilon}}\right)}\right)\ ,\ \forall i = 1,\dots,n\ , \\
\nabla^{2}_{f}Q_{\epsilon}^{semi}(f)_{ii} = \frac{-1}{\epsilon}\sum_{s=1}^{m}\left(\alpha_{i}\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\left(1 - \alpha_{i}\left(\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\right)\beta_{s}\ ,\ \forall i =1,\dots,n\ , \\
\nabla^{2}_{f}Q_{\epsilon}^{semi}(f)_{ij} = \frac{1}{\epsilon}\sum_{s=1}^{m}\alpha_{i}\alpha_{j}\left(\exp\left(\frac{f_{i}+g(f,C,\epsilon)_{s}-C_{is}}{\epsilon}\right)\right)\left(\exp\left(\frac{f_{j}+g(f,C,\epsilon)_{s}-C_{js}}{\epsilon}\right)\right)\beta_{s}\ ,\ \forall i \neq j = 1,\dots,n\ .
$$

## I. Semidual damped Newton (Direct inversion / No preconditioning)

In [ ]:

epsilons = [ 0.1, 0.05, 0.03 ]
rho = 0.95
c = 0.5
Semi_dual_dampedNewtonP   = []
results_DampedNewtonsemidual  = []
times_DampedNewtonsemidual     = []
Hessians_DampedNewtonsemidual  = []

#Cost matrix
C = distmat(x,y)
# a and b
a = normalize(np.ones(N[0]))
a = a.reshape(a.shape[0],-1)
b = normalize(np.ones(N[1]))
b = b.reshape(b.shape[0],-1)
for eps in epsilons:
    K = np.exp(-C/eps)
    # Line Search
    print("Semi-dual damped Newton for epsilon="+str(eps)+":")   
    f = a
    print("Doing for (",N[0],N[1],").")
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_SemiDual_np(C,a,b,f,eps,rho,c)
    out = Optimizer._update(maxiter = 50, debug = True)
    results_DampedNewtonsemidual.append(out)
    end = time.time()
    times_DampedNewtonsemidual.append(1e3*(end-start))
    print( " |- Computing P")
    Semi_dual_dampedNewtonP.append(GetP(np.exp(out['potential_f']/eps),K,np.exp(out['potential_g']/eps)))
    print( " |- Recording (unstabilized) Hessian \n")
    mat  = -eps*Optimizer.Hessian
    diag = 1/np.sqrt( np.vstack( a ) ).flatten()
    mat = diag*mat*diag
    Hessians_DampedNewtonsemidual.append( mat )

In [ ]:
plt.figure(figsize = (12,5))
plt.title("$$")
plt.title("$||P1 -a||_1+||P^T 1 -b||_1$")

for i in range(len(results_DampedNewtonsemidual)):
  error = np.asarray(results_DampedNewtonsemidual[i]['error'])
  plt.plot( error, label = 'Semi-dual damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Error in log-scale")
plt.legend(loc = "upper right")
plt.yscale( 'log' )
plt.tight_layout()
plt.savefig("../Images/DampedNewton_SemiDual_images_numpy/ErrorLinesearchNewton.pdf", format = 'pdf')
plt.show()
print("\n Error plots can increase! The error is not the objective function!")

In [ ]:
plt.figure(figsize = (12,5))
plt.title("$$")

plt.title("Objective Function")

for i in range(len(results_DampedNewtonsemidual)):
  value = np.asarray(results_DampedNewtonsemidual[i]['objectives'])
  plt.plot( value,label='Semi-dual damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)

plt.xlabel("Number of iterations")
plt.ylabel("Objective value")
plt.yscale('log')
plt.legend()
plt.savefig("../Images/DampedNewton_SemiDual_images_numpy/ObjectiveLineSearchNewton.pdf", format = 'pdf')
plt.show()

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Alpha")
for i in range(len(results_DampedNewtonsemidual)):
  plt.plot( np.asarray(results_DampedNewtonsemidual[i]["linesearch_steps"]),label='Damped Newton for $\epsilon=$'+ str(epsilons[i]), linewidth = 2)
plt.xlabel("Number of iterations")
plt.ylabel("Alpha in log-scale")
plt.legend()
# plt.yscale( 'log')
plt.savefig("../Images/DampedNewton_SemiDual_images_numpy/AlphaLineSearchNewton.pdf", format = 'pdf')
plt.show()

## II. Semidual damped Newton with preconditioning
Here we proceed similar to semi-dual damped Newton but with preconditioning. We consider $t$ eigenvalues of the Hessian and form the following preconditioning matrix:
$$
P = \left(I_{n+m}-\sum_{i-1}^{t}\left(1 - \frac{1}{\sqrt{\lambda_{i}}}\right)y_{i}y_{i}^{T}\right)\ .
$$
Now, at the $k^{th}$ iteration we solve the following equation:
$$
(P\nabla^{2}Q_{\epsilon}(f,g)P)(Pp_{k})=P\nabla Q_{\epsilon}(f,g)\ ,
$$
using iterative inversion methods such as "Conjugate gradient" and "GMRES" to get the update direction $p_{k}$, following which we use the Armijo condition to obtain the step size $\alpha_{k}$.

In [ ]:
def print_spectral_statistics(mat, stabilize=False):
    if stabilize:
        # Stabilizing largest and smallest eigenvalue
        min_vector = np.hstack( (np.ones(N[0])) )
        max_vector = np.hstack( (np.ones(N[0])) )
        norm = np.sqrt( N[0] )
        min_vector = min_vector/norm
        max_vector = max_vector/norm
        min_vector = min_vector.reshape( (min_vector.shape[0], 1) )
        max_vector = max_vector.reshape( (max_vector.shape[0], 1) )
        #
        mat = mat + np.dot( min_vector, min_vector.T)
        mat = mat - np.dot( max_vector, max_vector.T)
    # endif
    eig, v = np.linalg.eigh( mat )
    sorting_indices = np.argsort(eig)
    eig = eig[sorting_indices]
    v   = v[:, sorting_indices]
    
    #print( "Mean eigenvalue: ", np.mean(eig) )
    print( "List of smallest eigenvalues: ", eig[:10])
    print( "List of largest  eigenvalues: ", eig[-10:])
    min_index = np.argmin(eig)
    max_index = np.argmax(eig)
    min_value = eig[min_index]
    max_value = eig[max_index]
    min_vector = v[:, min_index]
    min_vector = min_vector/min_vector[0]
    max_vector = v[:,max_index]
    max_vector = max_vector/max_vector[0]
    condition_number = max_value/min_value
    # Test smallest and largest
    # print( "Min eigenvalue vector: ", min_vector)
    # print( "Max eigenvalue vector: ", max_vector)
    #
    #print( v[:,0]*np.sqrt( self.N1 + self.N2))
    #vector = v[:,0]
    #test = np.dot( result, vector)
    #print( np.linalg.norm(test) )
    #print("Min absolute eigenvalues: ", min_value)
    #print("Norm of v-1: ", np.linalg.norm(min_vector-eig_vector))
    print("Condition number: ", condition_number)
    # plt.hist( eig, 50)
    # plt.title( "Histogram of eigenvalues for Hessian")
    # plt.xlabel( "Eigenvalues")
    # plt.yscale( "log" )
    # plt.show()
    return eig,v

In [ ]:
eigs=[]
eigvecs=[]
for i in range(len(epsilons)):
    eps = epsilons[i]
    print("Spectral statistics of Hessian for epsilon="+str(eps))
    Hessian = Hessians_DampedNewtonsemidual[i]
    ev=print_spectral_statistics( Hessian, stabilize=False)
    eigs.append(ev[0])
    eigvecs.append(ev[1])
    print("")


In [ ]:
fig,ax=plt.subplots(figsize=(5,9), nrows=len(epsilons), ncols=1, sharey=True)
plt.title("Histogram of eigenvalues.")
for i in range(len(epsilons)):
    ax[i].hist( eigs[i], 50)
    ax[i].set_title( " $\epsilon$: "+str(epsilons[i]))
    ax[i].set_xlabel("Eigenvalues")
    ax[i].set_yscale( "log" )
    ax[i].set_xlim( 0, 1 )
plt.subplots_adjust(wspace=0,hspace=0)
plt.tight_layout()
plt.savefig("../Images/DampedNewton_SemiDual_images_numpy/eigenhistunstabilized.pdf", format = 'pdf')
plt.show()

In [ ]:
def build_preconditioners( num_eigs,modified_Hessian, ansatz=True ):
    # Diagonalize
    eigenvalues, eigenvectors = np.linalg.eigh( modified_Hessian )
    sorting_indices = np.argsort( eigenvalues )
    eigenvalues  = eigenvalues[sorting_indices]
    eigenvectors = eigenvectors[:, sorting_indices]
    # Form null vector
    if not ansatz:
        null_vector = eigenvectors[:, 0]
    else:
        null_vector = np.hstack( (np.ones(N[0])) )
        norm = np.sqrt( N[0])
        null_vector = null_vector/norm
    # Form other vectors
    indices = []
    for i in range(num_eigs):
        indices.append(i+1)
   
    precond_vectors = eigenvectors[:, indices ]
    precond_vectors = []
    for index in indices:
        precond_vectors.append( eigenvectors[:,index] )
    #
    return null_vector, precond_vectors

In [ ]:
num_eigs = 13
null_vector, precond_vectors = build_preconditioners( num_eigs, Hessians_DampedNewtonsemidual[-1], ansatz=False )

In [ ]:
rho = 0.95
c = 0.5
reset_starting_point    = True  
final_modified_Hessians = []
Semi_dual_dampedNewtonP           = []
results_DampedNewton_with_precodonditioner_SemiDual    = []
times_DampedNewton_with_precodonditioner_SemiDual     = []
precond_epsilons = [ 1.0, 0.5, 0.05, 0.03, 0.01 ]
f, g = None, None
# Cost matrix
C = distmat(x,y)
# a and b
a = normalize( np.ones(N[0]) )
a = a.reshape( a.shape[0],-1 )
b = normalize( np.ones(N[1]) )
b = b.reshape( b.shape[0],-1 )
for eps in epsilons:
    # Line Search
    print( "Semi-dual damped Newton for epsilon="+str(eps)+":" )    
    if (f is None) or (g is None):
        f, g = a*0, b*0
    print( "Doing for (",N[0],N[1],")." )
    print( " |- Iterating" )  
    start = time.time()
    Optimizer = computational_OT.DampedNewton_with_precodonditioner_SemiDual_np( C, a, b, f, g, eps, rho, c, null_vector, precond_vectors[:] )
    out = Optimizer._update( maxiter = 50, iterative_inversion = 30, version = None, debug = False, optType = 'cg' )
    results_DampedNewton_with_precodonditioner_SemiDual.append( out )
    end = time.time()
    times_DampedNewton_with_precodonditioner_SemiDual.append(1e3*(end-start))
    print( " |- Computing P" )
    if not reset_starting_point:
        f = Optimizer.x[:a.shape[0]]
        g = Optimizer.x[a.shape[0]:]
        # f = f.reshape( f.shape[0], -1)
        # g = g.reshape( g.shape[0], -1)
    Semi_dual_dampedNewtonP.append( GetP(np.exp(out['potential_f']/eps),np.exp(-C/eps),np.exp(out['potential_g']/eps)) )
    final_modified_Hessians.append( Optimizer.modified_Hessian )

In [ ]:
plt.figure(figsize = (20,7)) 
plt.title("$$") 
plt.title("$||P1 -a||_1+||P^T1 -b||_1$") 
for i in range(len(results_DampedNewton_with_precodonditioner_SemiDual)): 
  error = np.asarray(results_DampedNewton_with_precodonditioner_SemiDual[i]['error'] ) 
  plt.plot( error, label = 'Semi-dual damped Newton for $\epsilon=$'+ str(precond_epsilons[i]), linewidth = 2) 
plt.xlabel("Number of iterations")  
plt.ylabel("Error in log-scale")  
plt.legend() 
plt.yscale('log') 
plt.savefig("../Images/DampedNewton_SemiDual_images_numpy/ErrorDampedNewtonwithPrecond_final_cg.png") 
plt.show() 
print("\n Error plots can increase! The error is not the objective function!") 

In [ ]:
plt.figure(figsize = (20,7))
plt.subplot(2,1,1),
plt.title("Alpha")
for i in range(len(results_DampedNewton_with_precodonditioner_SemiDual)):
  plt.plot( np.asarray(results_DampedNewton_with_precodonditioner_SemiDual[i]["linesearch_steps"]),label='Damped Newton for $\epsilon=$'+ str(precond_epsilons[i]), linewidth = 2)
plt.xlabel("Number of iterations")
plt.ylabel("Alpha in log-scale")
plt.legend()
# plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize = (12,5))
plt.title("$$")
plt.title("Objective Function")
for i in range(len(results_DampedNewton_with_precodonditioner_SemiDual)):
  value = np.asarray(results_DampedNewton_with_precodonditioner_SemiDual[i]['objectives'])
  plt.plot( value,label='Damped Newton for $\epsilon=$'+ str(precond_epsilons[i]), linewidth = 2)
plt.xlabel("Number of iterations")
plt.ylabel("Objective value")
plt.yscale('log')
plt.legend()
plt.show()

In [ ]:
text = [
        "Preconditioning 1: Form E data",
        "Preconditioning 2: Form P data",
        "Form preconditioning functions",
        "Invert the linear system for p_k",
        "Unwinding",
        "Complete code block"
        ]

plt.figure( figsize = (20,10) )  
for j in range( len(results_DampedNewton_with_precodonditioner_SemiDual[0]['timings'][0]) ):
  values = []
  for i in range( len(results_DampedNewton_with_precodonditioner_SemiDual) ):
    mean = 0
    for k in range( len(results_DampedNewton_with_precodonditioner_SemiDual[i]['timings']) ):
      mean += results_DampedNewton_with_precodonditioner_SemiDual[i]['timings'][k][j]
    mean = mean/len(results_DampedNewton_with_precodonditioner_SemiDual[i]['timings']) 
    values.append( mean )
  if len(precond_epsilons) == len(values):
    plt.plot( precond_epsilons, np.asarray(values), label=text[j],linewidth = 2 )
    plt.legend( loc='upper left' )
plt.xlabel( "Epsilons" )
plt.ylabel( "Time in ms" )
plt.show()